In [1]:
import pandas as pd
import random
import numpy as np
import copy
from numba import njit


class Deck():
    def __init__(self, num_deck=1, is_black_jack = True):
        # num_decks = 0 means infinite number of decks and the pick up is with replacement
        self.contents = [(2, 'Diamonds'), (2, 'Hearts'), (2, 'Clubs'), (2, 'Spades'), (3, 'Diamonds'), (3, 'Hearts'),\
                         (3, 'Clubs'), (3, 'Spades'), (4, 'Diamonds'), (4, 'Hearts'), (4, 'Clubs'), (4, 'Spades'),\
                         (5, 'Diamonds'), (5, 'Hearts'), (5, 'Clubs'), (5, 'Spades'), (6, 'Diamonds'), (6, 'Hearts'),\
                         (6, 'Clubs'), (6, 'Spades'), (7, 'Diamonds'), (7, 'Hearts'), (2, 'Clubs'), (7, 'Spades'),\
                         (8, 'Diamonds'), (8, 'Hearts'), (8, 'Clubs'), (8, 'Spades'), (9, 'Diamonds'), (9, 'Hearts'),\
                         (9, 'Clubs'), (9, 'Spades'), (10, 'Diamonds'), (10, 'Hearts'), (10, 'Clubs'), (10, 'Spades'),\
                         (11, 'Diamonds'), (11, 'Hearts'), (11, 'Clubs'), (11, 'Spades'), (12, 'Diamonds'),\
                         (12, 'Hearts'), (12, 'Clubs'), (12, 'Spades'), (13, 'Diamonds'), (13, 'Hearts'),\
                         (13, 'Clubs'), (13, 'Spades'), (1, 'Diamonds'), (1, 'Hearts'), (1, 'Clubs'),\
                         (1, 'Spades')]
        if num_deck is not 0:
            self.decks = [self.contents for _ in range(num_deck)]
            self.decks = list(np.concatenate(self.decks, axis=0))
        else:
            self.decks = list(self.contents)
        self.num_deck = num_deck
        self.is_black_jack = is_black_jack

    def shuffle(self):
        # shuffle the cards
        random.shuffle(self.decks)
    
    def reset(self):
        self.decks = list(self.contents)
        random.shuffle(self.decks)

    def pop(self):
        # pop the top card out and hand it to the player
        card = self.decks.pop(0)
#         if self.is_black_jack and int(card[0])>10:
#             card = (10, card[1])
        # for num_deck = 0, pick up with replacement
        if self.num_deck == 0:
            self.reset()
        return card
    
# card_possibility = {1:4/52, 2:4/52, 3:4/52, 4:4/52, 5:4/52, 6:4/52, 7:4/52, 8:4/52, 9:4/52, 10:16/52}

In [2]:
@njit
def total(thehand, new_card):
    hand = np.copy(thehand)
    if hand[2] > 0:
        if new_card == 1 or new_card == 11:
            hand[2] +=1
            if hand[1] + hand[2] >= 12:
                return np.array([hand[1] + hand[2], hand[1], hand[2]])
            else:
                return np.array([hand[1] + hand[2] + 10, hand[1], hand[2]])
        else: 
            if hand[1] + hand[2] + new_card >= 12:
                return np.array([hand[1] + hand[2] + new_card, hand[1] + new_card, hand[2]])
            else:
                return np.array([hand[1] + hand[2] + 10 + new_card, hand[1] + new_card, hand[2]])
    
    else:
        if new_card == 1 or new_card == 11:
            if hand[1] >= 11: 
                hand[0] += 1
            else: 
                hand[0]+= 11
            return np.array([hand[0], hand[1], 1])
        else:
            return np.array([hand[0] + new_card,hand[1] + new_card,0])
#     for card in hand:
#         if card >=10:
#             total+= 10
#         elif card == 1:
#             if total >= 11: 
#                 total+= 1
#             else: 
#                 total+= 11
#         else:
#             total += card
#     return total

@njit
def card_dic_2_possibility(card_list):
#     card_possibility = np.array([4/52, 4/52, 4/52,4/52, 4/52, 4/52, 4/52, 4/52, 4/52, 4/52, 4/52, 4/52, 4/52])
    card_possibility = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
    num_of_cards = np.sum(card_list)
    card_possibility = card_list/num_of_cards
#     for i in range(len(card_list)):
#         card_possibility[i] = card_list[i]/num_of_cards
    return card_possibility
@njit     
def card_list_2_possibility(card_list):
#     card_possibility = np.array([4/52, 4/52, 4/52,4/52, 4/52, 4/52, 4/52, 4/52, 4/52, 16/52])
    card_possibility = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
    ori_card_possibility = card_dic_2_possibility(card_list)
    card_possibility[:9] = ori_card_possibility[:9]
    card_possibility[9] = np.sum(ori_card_possibility[9:])
    return card_possibility 
@njit
def state_transition_pos(state, next_state, card_possibility):
    if next_state - state==11:
        return card_possibility[0]
    elif (next_state - state)<11:
        return card_possibility[next_state-state-1]
    else:
        return 0
@njit    
def explosion_pos(state, card_possibility):
    the_indexs = [the_key for the_key in range(len(card_possibility)) if (the_key+1) > (21 - state)]
    possibility = 0
    for the_key in the_indexs:
        possibility += card_possibility[the_key]
    return possibility
        
    
# class agent():
#     def __init__(self):
#         self.hand=[]
#         self.card_dic={1:4, 2:4, 3:4, 4:4, 5:4, 6:4, 7:4, 8:4, 9:4, 10:4, 11:4, 12:4, 13:4}
        
#     def hit(self, card):
#         self.card_dic[card[1]] -=1
#         self.hand.append(card)
#         return hand

In [3]:
full_deck = np.array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1])
print(state_transition_pos(10, 20, card_list_2_possibility(full_deck)))

0.2653061224489796


In [28]:
@njit
def dealer_tree(root, total_value, new_card,probability_father, card_list):
    the_hand_value = total(total_value, root)[0]
    if the_hand_value >= 17:
        return_probability = np.array([0.0,0.0,0.0,0.0,0.0,0.0])
        index = new_card
        if the_hand_value > 21:
            return_probability[-1] = probability_father
        else:
            return_probability[(the_hand_value - 17)] = probability_father
#             print(return_probability[(root + total_value - 17)], probability_father)
        return return_probability          
    else:

        the_sum = np.array([0.0,0.0,0.0,0.0,0.0,0.0])
        for i in range(1, 14):
            temp_card_list = np.copy(card_list)
            if temp_card_list[i-1] == 0:
                continue
            if i < 10:
                index =i
#                 if i == 1 and root + total_value + 11 <= 21:
#                     index = 11
            else:
                index = 10
            p = probability_father * card_dic_2_possibility(card_list)[i-1]
#             p = probability_father * card_dic_2_possibility(np.array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]))[i-1]  ##### no memory
            temp_card_list[i-1] -= 1 
#             print(index, total_value, total(total_value, index))
            the_sum += dealer_tree(root, total(total_value, index), i, p, temp_card_list)
        return the_sum

In [5]:
full_deck = np.array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])
known = 10
print(dealer_tree(known, np.array([0,0,0]) , 0, 1, full_deck))

[0.11143196 0.10995077 0.111661   0.34101459 0.1118817  0.21405999]


In [7]:
for known in range(1,12):
    the_list = np.array([0.0,0.0,0.0,0.0,0.0,0.0])
    full_deck = np.array([4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4])
    full_deck[known-1] -= 1
    print(dealer_tree(known, np.array([0,0,0]) , 0, 1, full_deck))

[0.11816107 0.12256476 0.13236573 0.12359413 0.38971398 0.11360033]
[0.12769894 0.13312523 0.12865042 0.12755088 0.12809499 0.35487955]
[0.08318501 0.12328761 0.12569132 0.12545589 0.12996595 0.41241422]
[0.13004876 0.06505619 0.11205138 0.11804589 0.12603421 0.44876357]
[0.11766794 0.1214776  0.06914355 0.09425252 0.11715002 0.48030838]
[0.16842942 0.10341562 0.10419467 0.0511725  0.10407575 0.46871204]
[0.38965497 0.13867101 0.06889391 0.07091764 0.05272332 0.27913915]
[0.12778082 0.3798829  0.12544139 0.06290483 0.07188368 0.23210638]
[0.12727283 0.09827032 0.37367355 0.11742454 0.06200464 0.22135412]
[0.11061144 0.11782163 0.11027927 0.34280663 0.11927468 0.19920635]
[0.12494307 0.12593915 0.13566292 0.12756555 0.37247386 0.11341545]


|  | $17 $ | $ 18 $ | $ 19 $ |$ 20 $ | $ 21 $ | $ >21 $ 
| :-: | :-: | :-: | :-: | :-: | :-: | :-: | 
|$D$ = 1|0.13686463950085448 | 0.13682908636338428 | 0.12838842017234117 | 0.12492734458098267 | 0.16720358621088036 | 0.3057869231715582
|$D$ = 2| 0.13473832373192635 | 0.12731579456808595 | 0.1272707613455436 | 0.11929392965739376 | 0.11597212811067174 | 0.3754090625863796
|$D$ = 3| 0.12643738276854233 | 0.12657180965095952 | 0.11928607274643292 | 0.11885126743866382 | 0.11140841971210734 | 0.39744504768329647
|$D$ = 4| 0.1269431864804342 | 0.10999597787890625 | 0.11701622176097631 | 0.11205456544336158 | 0.11087950927876979 | 0.42311053915755337
|$D$ = 5| 0.1158896813007152 | 0.11926595396005142 | 0.11257121250092281 | 0.10049713278073259 | 0.10239210588317779 | 0.4493839135744026
|$D$ = 6| 0.16694766812059422 | 0.10645402905927319 | 0.10719186053213829 | 0.1007049316666462 | 0.09787847663196507 | 0.4208230339893839
|$D$ = 7| 0.3723448664078644 | 0.1385833815487343 | 0.07733443127482263 | 0.0788966658716112 | 0.07298698778514384 | 0.2598536671118236
|$D$ = 8| 0.13085746416975672 | 0.36298936571236573 | 0.12944463829318353 | 0.0682897637624276 | 0.06979142147145628 | 0.23862734659081034
|$D$ = 9| 0.12188622720133381 | 0.10392095413558577 | 0.35739125005537214 | 0.12225027908172217 | 0.06110882959412567 | 0.23344245993186055
|$D$ = 10| 0.1144181928090385 | 0.11287896102011571 | 0.11466240520259907 | 0.32887909897230555 | 0.11489750633525073 | 0.21426383566069046
|$D$ = 11| 0.1128628472665662 | 0.1144181928090385 | 0.11313103964990251 | 0.11472367670472444 | 0.32909742620452437 | 0.21576681736524406


### Zero Memory

|  | $17 $ | $ 18 $ | $ 19 $ |$ 20 $ | $ 21 $ | $ >21 $ 
| :-: | :-: | :-: | :-: | :-: | :-: | :-: | 
|$D$ = 1|0.13853324706661213 | 0.13557625054021205 | 0.12959686209249507 | 0.12318045055860413 | 0.17165838918106313 | 0.3014548005610004
|$D$ = 2| 0.13389323624407973 | 0.12883065521759768 | 0.1260741786224704 | 0.12040621085972823 | 0.11433079465779937 | 0.3764649243983222
|$D$ = 3| 0.12837581599698272 | 0.12541720014374658 | 0.12044133296672851 | 0.11732714721750172 | 0.11163452039002349 |0.39680398328501354
|$D$ = 4| 0.12604716704099472 | 0.1119239390019091 | 0.11587961410428384 | 0.1125499175834215 | 0.10940280936099833 | 0.42419655290839475
|$D$ = 5| 0.11836031977506437 | 0.1181926216768421 | 0.11331006273683414 | 0.09917874602838665 | 0.10277064578182889 | 0.4481876040010462
|$D$ = 6| 0.16660540908326865 | 0.10674332959738149 | 0.10622833250808146 | 0.10138349222335077 | 0.09649745705994037 |0.42254197952797856
|$D$ = 7| 0.3738770616642109 | 0.13846232675833037 | 0.07823470011427316 | 0.07808496034928418 | 0.0736038707074383 | 0.2577370804064635
|$D$ = 8| 0.13081526038571548 | 0.36457150798279353 | 0.12927739033573496 | 0.06905922386884035 | 0.0689196862556933 | 0.23735693117122297
|$D$ = 9| 0.12338003684683549 | 0.1037721937095806 | 0.35867415449389434 | 0.12184216679685489 | 0.0617446175887824 | 0.23058683056405277
|$D$ = 10| 0.11440694325797474 | 0.11440694325797474 | 0.11440694325797474 | 0.33009321776777867 | 0.11440694325797483 | 0.21227900920032267
|$D$ = 11| 0.11440694325797474 | 0.11440694325797474 | 0.11440694325797474 | 0.11440694325797474 | 0.33009321776777867 | 0.21227900920032267



In [29]:
elinamiate_all = False
@njit
def player_decision_mechanism(current_state, dealer_hand, card_dic): 
    dealer_list = dealer_tree(dealer_hand, np.array([0,0,0]) , 0, 1, card_dic)
#     print(sum1, sum2, sum3, sum4, sum5, sum6)
#     print(card_dic)
    card_possibility = card_list_2_possibility(card_dic)
#     print(card_possibility)
    player_explosion = explosion_pos(current_state, card_possibility)
    dealer_explosion = dealer_list[-1]
    the_sum = 0
    for next_state in range(17,22):
#         print(next_state, dealer_larger)
        temp_sum = 0
        for cal_state in range(next_state + 1,22):
            temp_sum += state_transition_pos(current_state, cal_state, card_possibility)
            
        temp_value = dealer_list[(next_state-17)] *(2*temp_sum + state_transition_pos(current_state, next_state, card_possibility))
        the_sum += temp_value
    expected_return = the_sum - 2 * player_explosion * dealer_explosion
    return 1 if expected_return>0 else 0

@njit
def player_stand_return(current_state, dealer_hand, card_dic): 
    dealer_list = dealer_tree(dealer_hand, np.array([0,0,0]) , 0, 1, card_dic)
    dealer_explosion = dealer_list[-1]
    the_return = dealer_explosion*2 -1
    return the_return

@njit
def player_hit_return(current_state, dealer_hand, card_dic): 
    dealer_list = dealer_tree(dealer_hand, np.array([0,0,0]) , 0, 1, card_dic)
#     print(sum1, sum2, sum3, sum4, sum5, sum6)
#     print(card_dic)
    card_possibility = card_list_2_possibility(card_dic)
    player_explosion = explosion_pos(current_state, card_possibility)
    dealer_explosion = dealer_list[-1]
    smaller_sum = 0
    for next_state in range(current_state + 1,17):
        smaller_sum += state_transition_pos(current_state, next_state, card_possibility)
#         print(next_state, state_transition_pos(current_state, next_state, card_possibility))
    sum_part = smaller_sum * (dealer_explosion*2-1) - player_explosion
    
    
    the_sum = 0
    for next_state in range(17,22):
        
        time_poss = state_transition_pos(current_state, next_state, card_possibility)
        dealer_larger = np.sum(dealer_list[(next_state-16):-1])
        dealer_smaller = np.sum(dealer_list[:next_state-17])
#         print(next_state, dealer_larger)
#         temp_value = dealer_list[(next_state-17)] *(2*temp_sum + state_transition_pos(current_state, next_state, card_possibility))
        the_sum += time_poss*(dealer_explosion + dealer_smaller - dealer_larger)
    expected_return = the_sum + sum_part
    return expected_return
    

In [30]:
full_deck = np.array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

pri_list = []
pri_list.append(player_hit_return(12, 2, full_deck)-\
                        player_stand_return(12, 2, full_deck))
print(pri_list)

[0.040354563405869126]


In [31]:
full_deck = np.array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

for player_state in range(12,17):
    pri_list = []
    for dealer_hand in range(1,22):
        pri_list.append(player_hit_return(player_state, dealer_hand, full_deck)-\
                        player_stand_return(player_state, dealer_hand, full_deck))
    print(pri_list)

[0.19522914370671507, 0.040354563405869126, 0.017487487085167397, -0.001576134718877159, -0.02773729075878259, -0.01570219362972483, 0.2195755652511978, 0.20128267626588214, 0.171349597373886, 0.13486873485388112, 0.19522914370671507, -0.10049577227340642, -0.1376503264027504, -0.17060447847778162, -0.20257570483814819, -0.23076923076923078, 0.6923076923076923, 0.5384615384615384, 0.3846153846153846, 0.23076923076923073, 0.07692307692307687]
[0.17707644669349543, -0.01383893138942427, -0.040100510748892926, -0.062068003469186506, -0.09192217281718429, -0.08058525339571457, 0.1788853049225098, 0.16347087211634215, 0.1357847230545246, 0.10193642911023154, 0.17707644669349543, -0.17509848909978196, -0.21790983849160964, -0.25597865181575585, -0.292841396913795, -0.3254437869822486, 0.6923076923076923, 0.5384615384615384, 0.3846153846153846, 0.23076923076923073, 0.07692307692307687]
[0.15892374968027578, -0.06803242618471766, -0.09768850858295325, -0.12255987221949594, -0.15610705487558596

In [32]:
full_deck = np.array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

for player_state in range(12,17):
    pri_list = []
    for dealer_hand in range(1,12):
        pri_list.append(player_decision_mechanism(player_state, dealer_hand, full_deck))
    print(pri_list)

[1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1]
[1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
[1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
[1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
[1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]


In [35]:
is_memorised = True
is_dealer_known = False
fold_cards = 0
num_of_decks= 1
total_counter = 0
# start_card = 6
theDeck = Deck()
# full_deck = {1:4, 2:4, 3:4, 4:4, 5:4, 6:4, 7:4, 8:4, 9:4, 10:4, 11:4, 12:4, 13:4}
full_deck = np.array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

dawn_counter = 0
game_counter = 0
win_counter = 0
lose_counter = 0

for i in range(int(1e7)):
    theDeck.reset()
#     print(theDeck.decks)
#     cur_deck = copy.deepcopy({1:4, 2:4, 3:4, 4:4, 5:4, 6:4, 7:4, 8:4, 9:4, 10:4, 11:4, 12:4, 13:4})
    cur_deck = np.copy([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])
    cur_deck = cur_deck * num_of_decks
#     print(cur_deck)
    for _ in range(fold_cards):
        fold_card = theDeck.pop()
#         print(int(fold_card[0]))
        cur_deck[int(fold_card[0])-1] -= 1 
#     print(cur_deck)
    player_state = np.array([0, 0, 0])
    dealer_state = np.array([0, 0, 0])
    for ii in range(2):
        the_card = theDeck.pop()
        cur_deck[int(the_card[0])-1] -= 1 
        if int(the_card[0]) < 10:
            index = int(the_card[0])
        else:
            index = 10
        player_state = total(player_state, index)
        
#         if int(the_card[0]) == 1 and player_state + 11 <= 21:
#             player_state += 11
#         elif int(the_card[0])> 10:
#             player_state += 10
#         else:
#             player_state += int(the_card[0])
    if is_dealer_known:
        for ii in range(2):
            dealer_card = theDeck.pop()
            cur_deck[int(dealer_card[0])-1] -= 1 
            if int(dealer_card[0]) < 10:
                index = int(dealer_card[0])
            else:
                index = 10
            dealer_state = total(dealer_state, index)
    else:
        dealer_card = theDeck.pop()
        cur_deck[int(dealer_card[0])-1] -= 1 
        if int(dealer_card[0]) < 10:
            index = int(dealer_card[0])
        else:
            index = 10
        dealer_state = total(dealer_state, index)
    
#     if int(dealer_card[0]) == 1 and dealer_state + 11 <= 21:
#         dealer_state = 11
#     elif int(dealer_card[0])> 10:
#         dealer_state = 10
#     else:
#         dealer_state = int(dealer_card[0])
    player_action = -1
    while(player_action !=0): 
#         print(i, player_action)
#         print(player_state)
        
        if player_state[0] < 12:
            player_action = 1
        elif player_state[0] > 16:
            player_action = 0
        else:
            if is_memorised:
                player_action = player_decision_mechanism(player_state[0], dealer_state[0], cur_deck)
            else:
                player_action = player_decision_mechanism(player_state[0], dealer_state[0], full_deck)
        if player_action == 1:  
            the_card = theDeck.pop() 
            cur_deck[int(the_card[0])-1] -= 1 
            if int(the_card[0]) < 10:
                index = int(the_card[0])
            else:
                index = 10
            player_state = total(player_state, index)
            
            
#             if int(the_card[0]) == 1 and player_state + 11 <= 21:
#                 player_state += 11
#             elif int(the_card[0])> 10:
#                 player_state += 10
#             else:
#                 player_state += int(the_card[0])
    if player_state[0] > 21:
        total_counter -= 1
        lose_counter += 1
    else:
        while(dealer_state[0]<17):
            the_card = theDeck.pop() 
            cur_deck[int(the_card[0])-1] -= 1 
            if int(the_card[0]) < 10:
                index = int(the_card[0])
            else:
                index = 10
            dealer_state = total(dealer_state, index)
            
#             if int(the_card[0]) == 1 and dealer_state + 11 <= 21:
#                 dealer_state += 11
#             elif int(the_card[0])> 10:
#                 dealer_state += 10
#             else:
#                 dealer_state += int(the_card[0])
        if dealer_state[0] > 21:
            total_counter += 1
            win_counter += 1
        elif dealer_state[0] > player_state[0]:
            total_counter -= 1
            lose_counter += 1
        elif dealer_state[0] < player_state[0]:
            total_counter += 1
            win_counter += 1
        elif dealer_state[0] == player_state[0]:
            dawn_counter += 1

print('win',win_counter,'lose',lose_counter,'dawn',dawn_counter)

win 4222 lose 4862 dawn 916


In [19]:
#with memory 20 cards
total_counter
print('win:',win_counter,'lose:',lose_counter,'dawn:',dawn_counter, 'win rate:', (win_counter)/(lose_counter+win_counter))

win: 4277153 lose: 4829513 dawn: 893334 win rate: 0.4696727649833649


In [33]:
#with memory 30 cards
total_counter
print('win:',win_counter,'lose:',lose_counter,'dawn:',dawn_counter, 'win rate:', (win_counter)/(lose_counter+win_counter))

win: 4552324 lose: 4556757 dawn: 890919 win rate: 0.4997566713919878


In [73]:
#with memory 20 cards
total_counter

-270902

In [75]:
#with memory 30 cards
total_counter

-196520

In [14]:
#without memory 1e7
total_counter

-340935

In [111]:
#with memory fold card
total_counter

-373

In [117]:
#without memory fold card
total_counter

-53

In [212]:
sampled_df = pd.read_csv('C:/Users/13862/Desktop/RA_BJ/sampled_data.csv')
sampled_df = sampled_df.drop(['Unnamed: 0'], axis=1)
sampled_list = sampled_df.values.tolist()
sampled_df

,dealer_player,deck
0,[14 10],[0 1 2 1 0 1 0 1 1 1 2 2 1]
1,[8 7],[1 2 1 1 0 2 1 1 1 0 2 0 1]
2,[17 6],[1 0 1 1 1 1 1 2 2 0 0 1 2]
3,[10 10],[0 2 2 2 0 1 0 1 1 2 1 0 1]
4,[13 10],[0 1 1 0 1 1 1 0 2 2 2 0 2]
...,...,...
49995,[19 11],[2 2 0 1 1 0 0 1 3 0 2 0 1]
49996,[16 11],[1 0 1 1 0 1 2 3 1 1 0 1 1]
49997,[14 10],[1 2 1 0 2 1 1 1 1 2 0 0 1]
49998,[10 7],[1 1 2 1 1 1 1 3 1 1 0 0 0]


In [213]:
hit_list = []
stand_list = []
for i in range(len(sampled_list)):
#     print(sampled_list[i][0],sampled_list[i][1])
    full_deck = np.array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])
    cur_deck = full_deck - np.array(sampled_list[i][1][1:-1].split()).astype(int)
    thecard_value = np.array(sampled_list[i][0][1:-1].split()).astype(int)
    if thecard_value[1] > 11:
        print('error')
    if thecard_value[0] < 17 and thecard_value[0] > 11:
        hit_return = player_hit_return(thecard_value[0], thecard_value[1], cur_deck) + 1
        stand_return = player_stand_return(thecard_value[0], thecard_value[1], cur_deck) + 1
        hit_list.append(hit_return/(hit_return+stand_return))
        stand_list.append(stand_return/(hit_return+stand_return))
    elif thecard_value[0] < 12:
        hit_list.append(1)
        stand_list.append(0)
    elif thecard_value[0] >= 17:
        hit_list.append(0)
        stand_list.append(1)

In [214]:
hit_list

[0.5828892008681399,
 1,
 0,
 1,
 0.5908547904696759,
 1,
 0,
 0.5827113637966445,
 0.5518188156905116,
 0,
 0.5765584858439664,
 0,
 0,
 0,
 0,
 0.504402537418084,
 0.517309509367665,
 0.4626710919658067,
 0,
 0.5640674716410193,
 0,
 0.5683020712569818,
 0.4720995636223394,
 0,
 1,
 1,
 0.5220512050988959,
 0.5899745591395711,
 0,
 0.46088052392144724,
 0.5901502496443947,
 1,
 1,
 1,
 0.4159382486821788,
 0,
 1,
 1,
 0,
 0.5120266205361744,
 0.4523776296264754,
 0,
 1,
 0.524001035380824,
 0.5114893621360448,
 0,
 0.5478464979104226,
 0.5985676534512344,
 0.5263703502756977,
 0,
 0,
 1,
 0,
 0.45738897612865254,
 1,
 0.4598301509694455,
 0,
 1,
 0.5160614081524616,
 0,
 0,
 0,
 0,
 0.4867241016299492,
 0.5672416447982257,
 0.4818572104305023,
 0,
 1,
 1,
 0,
 0.5627989851180345,
 0.5297910524908009,
 0,
 0.4485461935376005,
 1,
 1,
 0,
 1,
 0.42979388243794264,
 0.48751191825915813,
 0.36224926672767077,
 0.5318800305228195,
 0,
 0,
 1,
 0,
 0.6132936025684367,
 0.5873733236709048,


In [215]:
sampled_df['hit_prob'] = hit_list
sampled_df['stand_prob'] = stand_list

In [216]:
sampled_df.to_csv('C:/Users/13862/Desktop/RA_BJ/sampled_prob.csv')

In [217]:
sampled_df

,dealer_player,deck,hit_prob,stand_prob
0,[14 10],[0 1 2 1 0 1 0 1 1 1 2 2 1],0.582889,0.417111
1,[8 7],[1 2 1 1 0 2 1 1 1 0 2 0 1],1.000000,0.000000
2,[17 6],[1 0 1 1 1 1 1 2 2 0 0 1 2],0.000000,1.000000
3,[10 10],[0 2 2 2 0 1 0 1 1 2 1 0 1],1.000000,0.000000
4,[13 10],[0 1 1 0 1 1 1 0 2 2 2 0 2],0.590855,0.409145
...,...,...,...,...
49995,[19 11],[2 2 0 1 1 0 0 1 3 0 2 0 1],0.000000,1.000000
49996,[16 11],[1 0 1 1 0 1 2 3 1 1 0 1 1],0.489371,0.510629
49997,[14 10],[1 2 1 0 2 1 1 1 1 2 0 0 1],0.530964,0.469036
49998,[10 7],[1 1 2 1 1 1 1 3 1 1 0 0 0],1.000000,0.000000
